In [1]:
import numpy as np
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random
import gymnasium as gym
from gymnasium import spaces
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, TensorDataset
from collections import namedtuple, deque
from google.colab import drive
import networkx as nx
from collections import defaultdict
drive.mount('/content/drive')

Mounted at /content/drive


# Env

In [2]:
class FourRoomEnv(gym.Env):
    """
    Custom FourRoom environment with fixed start in upper-left room and goal in bottom-right room.
    Each room is 5x5, total grid 11x11 with walls separating rooms and hallways (gaps) connecting them.
    Reward: +1 only when reaching the goal, 0 otherwise.
    """
    def __init__(self):
        self.grid_size = 11
        self.action_space = spaces.Discrete(4)  # 0: up, 1: down, 2: left, 3: right
        self.observation_space = spaces.Discrete(self.grid_size ** 2)
        self.start_pos = (0, 0)  # Upper-left room
        self.goal_pos = (10, 10)  # Bottom-right room
        self.actions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # up, down, left, right

        # Define walls (impassable positions)
        self.walls = set()
        # Add all positions in row 5 as potential walls
        for c in range(self.grid_size):
            self.walls.add((5, c))
        # Add all positions in column 5 as potential walls
        for r in range(self.grid_size):
            self.walls.add((r, 5))

        # Remove the gap positions from the walls
        self.walls.remove((2, 5)) # Vertical gap top
        self.walls.remove((8, 5)) # Vertical gap bottom
        self.walls.remove((5, 2)) # Horizontal gap left
        self.walls.remove((5, 8)) # Horizontal gap right

        # Build graph for optimal policy (using networkx)
        self.positions = [(r, c) for r in range(self.grid_size) for c in range(self.grid_size) if (r, c) not in self.walls]
        self.G = nx.Graph()
        for pos in self.positions:
            self.G.add_node(pos)
        for pos in self.positions:
            r, c = pos
            for dr, dc in self.actions:
                nr, nc = r + dr, c + dc
                if (nr, nc) in self.positions:
                    self.G.add_edge(pos, (nr, nc))

        # Compute shortest path distances to goal
        self.dist_to_goal = nx.shortest_path_length(self.G, target=self.goal_pos)

        # Precompute optimal actions for each position
        self.optimal_actions = defaultdict(list)
        for pos in self.positions:
            if pos == self.goal_pos:
                continue
            # Ensure pos is in dist_to_goal before accessing (should be true if graph is connected)
            if pos in self.dist_to_goal:
                dist = self.dist_to_goal[pos]
                for a_idx, (dr, dc) in enumerate(self.actions):
                    nr, nc = pos[0] + dr, pos[1] + dc
                    if (nr, nc) in self.positions and self.dist_to_goal.get((nr, nc), float('inf')) == dist - 1:
                        self.optimal_actions[pos].append(a_idx)

        self.reset()

    def reset(self, *, seed=None, options=None):
        self.current_pos = self.start_pos
        return self.pos_to_state(self.current_pos), {}

    def pos_to_state(self, pos):
        return pos[0] * self.grid_size + pos[1]

    def state_to_pos(self, state):
        return divmod(state, self.grid_size)

    def step(self, action):
        row, col = self.current_pos
        dr, dc = self.actions[action]
        new_row, new_col = row + dr, col + dc
        if 0 <= new_row < self.grid_size and 0 <= new_col < self.grid_size and (new_row, new_col) not in self.walls:
            self.current_pos = (new_row, new_col)
        terminated = self.current_pos == self.goal_pos
        reward = 1 if terminated else 0
        truncated = False  # No time limit
        return self.pos_to_state(self.current_pos), reward, terminated, truncated, {}

    def get_action(self, pos, epsilon=0.3):
        """
        Epsilon-greedy policy based on optimal actions.
        With prob epsilon, random action; else, random optimal action.
        """
        if random.random() < epsilon or not self.optimal_actions[pos]:
            return random.randint(0, 3)
        return random.choice(self.optimal_actions[pos])

# Load Dataset

In [3]:
filename = f"drive/MyDrive/Colab Notebooks/LLMAgent/FourRoom/Datasets/FourRoom-Low.pkl"

In [4]:
with open(filename, "rb") as f:
  dataset = pickle.load(f)
print(f"Loaded {len(dataset)} trajectories from {filename}")

Loaded 1000 trajectories from drive/MyDrive/Colab Notebooks/LLMAgent/FourRoom/Datasets/FourRoom-Low.pkl


In [5]:
max_steps = 100

# Policy Network

In [6]:
class PolicyNetwork(nn.Module):
    def __init__(self, state_dim, goal_dim, action_dim):
        super(PolicyNetwork, self).__init__()
        self.fc1 = nn.Linear(state_dim + goal_dim, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, action_dim)
        self.softmax = nn.Softmax()


    def forward(self, state, goal):
        x = torch.cat([state, goal], dim=-1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.softmax(x)
        return x  # logits

# Train

In [25]:
def label_subtask(env, state):
    """
    Label sub-task based on which room the state belongs to:
      k=1 -> upper-left room
      k=3 -> bottom-right room
      k=2 -> other rooms or hallways
    """
    r, c = env.state_to_pos(state)

    # Upper-left room (rows 0-4, cols 0-4)
    if r < 5 and c < 5:
        return 1

    # Bottom-right room (rows 6-10, cols 6-10)
    if r > 5 and c > 5:
        return 3

    # Otherwise (upper-right, bottom-left, or hallways)
    return 2

In [50]:
# Train
env = FourRoomEnv()
state_dim = env.observation_space.n
action_dim = env.action_space.n
goal_dim = 3

policy_net = PolicyNetwork(state_dim, goal_dim, action_dim)
optimizer = optim.Adam(policy_net.parameters(), lr=3e-4)


# One-hot encoding for discrete actions
def action_to_tensor(action, action_dim):
  action = torch.tensor(action)
  tensor = F.one_hot(action, num_classes=action_dim).float()

  return tensor

In [51]:
# One-hot encoding for discrete states
def state_to_tensor(state, state_dim):
  state = torch.tensor(state)
  tensor = F.one_hot(state, num_classes=state_dim).float()

  return tensor

In [52]:
# One-hot encoding for discrete subgoals
def goal_to_tensor(goal, goal_dim=3):

  goal = goal - np.ones_like(goal)
  goal = torch.tensor(goal)
  tensor = F.one_hot(goal, num_classes=goal_dim).float()

  return tensor

In [53]:
# Training loop
num_epochs = 1000
batch_size = 256
losses = []

In [54]:
# Load Dataset into Buffer
gamma = 0.99
num_episodes = len(dataset)

state_dataset = []
action_dataset = []
goal_dataset = []

for i in range(num_episodes):
  epi_len = len(dataset[i])
  for j in range(epi_len):
    state = dataset[i][j][0]
    action = dataset[i][j][1]
    goal = label_subtask(env, state)

    state_dataset.append(state)
    action_dataset.append(action)
    goal_dataset.append(goal)

In [55]:
loss_fn = nn.BCELoss()
dataset_size = len(goal_dataset)  # assume all 3 lists are same length

In [57]:
reward_list = []
std_list = []
for epoch in range(num_epochs):
    # Sample indices (without replacement here)
    indices = np.random.choice(dataset_size, size=batch_size, replace=False)

  # Apply indices to all lists
    batch_goals = [goal_dataset[i] for i in indices]
    batch_states = [state_dataset[i] for i in indices]
    batch_actions = [action_dataset[i] for i in indices]

    #print(batch_goals)
    goal_tensor = goal_to_tensor(batch_goals)
    state_tensor = state_to_tensor(batch_states, state_dim)
    action_tensor = action_to_tensor(batch_actions, action_dim)

    # Forward

    logits = policy_net(state_tensor, goal_tensor)
    loss = loss_fn(logits, action_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 5 == 0:
      print(f"Epoch {epoch},  Policy Loss: {loss:.4f}")
      eval_rewards = []
      eval_length = []
      for _ in range(10):
        eval_r, eval_l = evaluate_policy(state_dim=state_dim)
        eval_rewards.append(eval_r)
        eval_length.append(eval_l)

      eval_rewards = np.array(eval_rewards)
      eval_length = np.array(eval_length)

      print(f"Average evaluation reward: {np.mean(eval_rewards):.2f} ± {np.std(eval_rewards):.2f}")
      print(f"Average Length: {np.mean(eval_length):.2f} ± {np.std(eval_length):.2f}")

      reward_list.append(np.mean(eval_rewards))
      std_list.append(np.std(eval_rewards))

Epoch 0,  Policy Loss: 0.5630
Average evaluation reward: 0.00 ± 0.00
Average Length: 100.00 ± 0.00
Epoch 5,  Policy Loss: 0.5633
Average evaluation reward: 0.00 ± 0.00
Average Length: 100.00 ± 0.00
Epoch 10,  Policy Loss: 0.5613
Average evaluation reward: 0.00 ± 0.00
Average Length: 100.00 ± 0.00
Epoch 15,  Policy Loss: 0.5606
Average evaluation reward: 0.00 ± 0.00
Average Length: 100.00 ± 0.00
Epoch 20,  Policy Loss: 0.5614
Average evaluation reward: 0.00 ± 0.00
Average Length: 100.00 ± 0.00
Epoch 25,  Policy Loss: 0.5609
Average evaluation reward: 0.00 ± 0.00
Average Length: 100.00 ± 0.00
Epoch 30,  Policy Loss: 0.5584
Average evaluation reward: 0.00 ± 0.00
Average Length: 100.00 ± 0.00
Epoch 35,  Policy Loss: 0.5629
Average evaluation reward: 0.00 ± 0.00
Average Length: 100.00 ± 0.00
Epoch 40,  Policy Loss: 0.5608
Average evaluation reward: 0.00 ± 0.00
Average Length: 100.00 ± 0.00
Epoch 45,  Policy Loss: 0.5588
Average evaluation reward: 0.00 ± 0.00
Average Length: 100.00 ± 0.00
Ep

# Eval

In [56]:
# Evaluation
def evaluate_policy(state_dim):
  state, _ = env.reset()
  total_reward = 0
  done = False
  step = 0
  while not done:
    goal = label_subtask(env, state)

    state_tensor = torch.FloatTensor(state_to_tensor(state, state_dim=state_dim))
    goal_tensor = torch.FloatTensor(goal_to_tensor(goal))
    state_tensor = state_tensor.unsqueeze(0)
    goal_tensor = goal_tensor.unsqueeze(0)

    action = policy_net(state_tensor, goal_tensor).argmax().item()
    state, reward, terminated, truncated, _ = env.step(action)
    done = terminated or truncated
    total_reward += reward
    step +=1

    if step >= max_steps:
      break


  return total_reward, step

In [58]:
reward_list

[np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float6